In [ ]:
hf_home_dir = "./hf_cache"

In [ ]:
import os

os.environ["HF_HOME"] = hf_home_dir

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

dataset = load_dataset("alexandrainst/nst-da")

In [ ]:
# num_samples = len(dataset)

In [ ]:
# import random

# random.seed(10)
# rands = random.sample(range(len(dataset)), num_samples)
# dataset = dataset.select(rands)

In [ ]:
from transformers import SpeechT5Processor

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)
tokenizer = processor.tokenizer

In [ ]:
dataset = dataset.filter(
    lambda x: not (set("0123456789") & set(x)), input_columns=["text"]
)

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


vocabs = dataset.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=dataset["train"].column_names,
)

dataset_vocab = set(vocabs["train"]["vocab"][0] + vocabs["test"]["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

In [ ]:
missing_vocab = dataset_vocab - tokenizer_vocab

In [ ]:
replacements = [
    ("&", "og"),
    ("\r", " "),
    ("´", ""),
    ("\\", ""),
    ("¨", " "),
    ("Å", "AA"),
    ("Æ", "AE"),
    ("É", "E"),
    ("Ö", "OE"),
    ("Ø", "OE"),
    ("á", "a"),
    ("ä", "ae"),
    ("å", "aa"),
    ("è", "e"),
    ("î", "i"),
    ("ô", "oe"),
    ("ö", "oe"),
    ("ø", "oe"),
    ("ü", "y"),
]

In [ ]:
def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["text"] = inputs["text"].replace(src, dst)
    return inputs


dataset = dataset.map(cleanup_text)

In [ ]:
from collections import defaultdict

speaker_counts = defaultdict(int)

for speaker_id in dataset["train"]["speaker_id"]:
    speaker_counts[speaker_id] += 1

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.hist(speaker_counts.values(), bins=20)
plt.ylabel("Speakers")
plt.xlabel("Examples")
plt.show()

In [ ]:
def select_speaker(speaker_id):
    return 280 <= speaker_counts[speaker_id] <= 327

In [ ]:
dataset["train"] = dataset["train"].filter(
    select_speaker,
    input_columns=["speaker_id"],
)

In [ ]:
len(set(dataset["train"]["speaker_id"]))

In [ ]:
len(dataset["train"])

In [ ]:
speaker_id_examples = [
    (k, v) for (k, v) in list(speaker_counts.items()) if 280 <= v <= 327
]
speaker_id_examples_sorted = sorted(
    speaker_id_examples, key=lambda x: x[1], reverse=True
)
speaker_id_examples_sorted[0:20]

In [ ]:
import os
import torch
from speechbrain.pretrained import EncoderClassifier

device = "cuda" if torch.cuda.is_available() else "cpu"

spk_model_name = "speechbrain/spkrec-xvect-voxceleb"

speaker_model = EncoderClassifier.from_hparams(
    source=spk_model_name,
    run_opts={"device": device},
    savedir=os.path.join("/tmp", spk_model_name),
)


def create_speaker_embedding(waveform):
    with torch.no_grad():
        speaker_embeddings = speaker_model.encode_batch(torch.tensor(waveform))
        speaker_embeddings = torch.nn.functional.normalize(speaker_embeddings, dim=2)
        speaker_embeddings = speaker_embeddings.squeeze().cpu().numpy()
    return speaker_embeddings

In [ ]:
from speechbrain.pretrained import SpectralMaskEnhancement

metricgan_model_name = "speechbrain/metricgan-plus-voicebank"


enhance_model = SpectralMaskEnhancement.from_hparams(
    source=metricgan_model_name,
    savedir=os.path.join("/tmp", metricgan_model_name),
    run_opts={"device": device},
)


def enhance_audio(waveform):
    tensor = torch.tensor(waveform).reshape(1, -1).float()
    enhanced = enhance_model.enhance_batch(tensor, lengths=torch.tensor([1.0]))
    enhanced = enhanced.squeeze().cpu().numpy()
    return enhanced

In [ ]:
from resemble_enhance.enhancer.inference import enhance
from IPython.utils import io


def mega_enhance_audio(
    waveform, sr, device="cuda", nfe=64, solver="midpoint", lambd=0.9, tau=0.95
):
    tensor = torch.tensor(waveform).float()
    with io.capture_output() as _:
        enhanced, new_sr = enhance(
            tensor, sr, device, nfe=nfe, solver=solver, lambd=lambd, tau=tau
        )
    enhanced_cpu = enhanced.cpu().numpy()
    return enhanced_cpu, new_sr

In [ ]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
assert sampling_rate == 16000
dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [ ]:
# dataset_id = "JackismyShephard/nst-da-norm"

# dataset["train"].push_to_hub(
#     dataset_id, split="train", commit_message="add train split"
# )

# dataset["test"].push_to_hub(dataset_id, split="test", commit_message="add test split")

In [ ]:
# from huggingface_hub import DatasetCard, DatasetCardData

# # Using the Default Template
# card_data = DatasetCardData(
#     size_categories="100K<n<1M",
#     license="cc0-1.0",
#     task_categories=["automatic-speech-recognition", "text-to-speech"],
#     language="da",
#     pretty_name="NST-da Normalized",
#     annotations_creators=["machine-generated", "expert-generated"],
#     language_creators=["expert-generated"],
#     multilinguality="monolingual",
#     source_datasets="extended",
# )
# card = DatasetCard.from_template(
#     card_data,
# )
# card.push_to_hub(dataset_id, commit_message="update dataset card")

In [ ]:
female_23_vestjylland = dataset["train"].filter(
    lambda x: x == 202, input_columns=["speaker_id"]
)[2]
female_24_storkoebenhavn = dataset["train"].filter(
    lambda x: x == 404, input_columns=["speaker_id"]
)[55]
female_49_nordjylland = dataset["train"].filter(
    lambda x: x == 419, input_columns=["speaker_id"]
)[1]
male_51_vest_sydsjaelland = dataset["train"].filter(
    lambda x: x == 475, input_columns=["speaker_id"]
)[1]
male_18_vest_sydsjaelland = dataset["train"].filter(
    lambda x: x == 83, input_columns=["speaker_id"]
)[17]
male_31_fyn = dataset["train"].filter(lambda x: x == 496, input_columns=["speaker_id"])[
    37
]

In [ ]:
female_22_oestjylland = dataset["train"].filter(
    lambda x: x == 301, input_columns=["speaker_id"]
)[0]
female_24_storkoebenhavn_2 = dataset["train"].filter(
    lambda x: x == 404, input_columns=["speaker_id"]
)[0]
female_44_nordjylland = dataset["train"].filter(
    lambda x: x == 517, input_columns=["speaker_id"]
)[0]

male_18_vest_syd_sjaelland = dataset["train"].filter(
    lambda x: x == 83, input_columns=["speaker_id"]
)[2]
male_31_fyn_2 = dataset["train"].filter(
    lambda x: x == 496, input_columns=["speaker_id"]
)[8]
male_55_storkoebenhavn = dataset["train"].filter(
    lambda x: x == 43, input_columns=["speaker_id"]
)[0]

In [ ]:
speaker_list = [
    (female_23_vestjylland, "female_23_vestjylland.npy"),
    (female_24_storkoebenhavn, "female_24_storkoebenhavn.npy"),
    (female_49_nordjylland, "female_49_nordjylland.npy"),
    (male_51_vest_sydsjaelland, "male_51_vest_sydsjaelland.npy"),
    (male_18_vest_sydsjaelland, "male_18_vest_sydsjaelland.npy"),
    (male_31_fyn, "male_31_fyn.npy"),
]

speaker_embeddings_list = [
    (create_speaker_embedding(enhance_audio(speaker["audio"]["array"])), file_name)
    for (speaker, file_name) in speaker_list
]

In [ ]:
speaker_list_2 = [
    (female_22_oestjylland, "female_22_oestjylland.npy"),
    (female_24_storkoebenhavn_2, "female_24_storkoebenhavn.npy"),
    (female_44_nordjylland, "female_44_nordjylland.npy"),
    (male_18_vest_syd_sjaelland, "male_18_vest_syd_sjaelland.npy"),
    (male_31_fyn_2, "male_31_fyn.npy"),
    (male_55_storkoebenhavn, "male_55_storkoebenhavn.npy"),
]

speaker_embeddings_list_2 = [
    (
        create_speaker_embedding(
            mega_enhance_audio(
                speaker["audio"]["array"], speaker["audio"]["sampling_rate"]
            )[0]
        ),
        file_name,
    )
    for (speaker, file_name) in speaker_list_2
]

In [ ]:
from pathlib import Path

root_metricgan_plus = "./embeddings/nst-da-metricgan-plus/"
root_resemble_enhance = "./embeddings/nst-da-resemble-enhance/"
Path(root_metricgan_plus).mkdir(parents=True, exist_ok=True)
Path(root_resemble_enhance).mkdir(parents=True, exist_ok=True)

In [ ]:
import numpy as np

for embedding, file_name in speaker_embeddings_list:
    np.save(root_metricgan_plus + file_name, embedding)

for embedding, file_name in speaker_embeddings_list_2:
    np.save(root_resemble_enhance + file_name, embedding)

In [ ]:
def prepare_dataset(example, enhance=False):
    audio = example["audio"]["array"]
    if enhance:
        audio = enhance_audio(audio)

    example = processor(
        text=example["text"],
        audio_target=audio,
        sampling_rate=sampling_rate,
        return_attention_mask=False,
    )

    # strip off the batch dimension
    example["labels"] = example["labels"][0]

    # use SpeechBrain to obtain x-vector

    example["speaker_embeddings"] = create_speaker_embedding(audio)

    return example

In [ ]:
processed_example = prepare_dataset(dataset["train"][0])
list(processed_example.keys())

In [ ]:
processed_example["speaker_embeddings"].shape

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(processed_example["labels"].T)
plt.show()

In [36]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset["train"].column_names)

In [ ]:
input_length_max = 600


def is_not_too_long(input_ids):
    input_length = len(input_ids)
    return input_length < input_length_max


dataset = dataset.filter(
    is_not_too_long,
    input_columns=["input_ids"],
)

In [ ]:
len(dataset["train"])

In [ ]:
len(dataset["test"])

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class TTSDataCollatorWithPadding:
    processor: Any

    def __call__(
        self, features: List[Dict[str, Union[List[int], torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        input_ids = [{"input_ids": feature["input_ids"]} for feature in features]
        label_features = [{"input_values": feature["labels"]} for feature in features]
        speaker_features = [feature["speaker_embeddings"] for feature in features]

        # collate the inputs and targets into a batch
        batch = processor.pad(
            input_ids=input_ids, labels=label_features, return_tensors="pt"
        )

        # replace padding with -100 to ignore loss correctly
        batch["labels"] = batch["labels"].masked_fill(
            batch.decoder_attention_mask.unsqueeze(-1).ne(1), -100
        )

        # not used during fine-tuning
        del batch["decoder_attention_mask"]

        # round down target lengths to multiple of reduction factor
        if model.config.reduction_factor > 1:
            target_lengths = torch.tensor(
                [len(feature["input_values"]) for feature in label_features]
            )
            target_lengths = target_lengths.new(
                [
                    length - length % model.config.reduction_factor
                    for length in target_lengths
                ]
            )
            max_length = max(target_lengths)
            batch["labels"] = batch["labels"][:, :max_length]

        # also add in the speaker embeddings
        batch["speaker_embeddings"] = torch.tensor(speaker_features)

        return batch

In [ ]:
data_collator = TTSDataCollatorWithPadding(processor=processor)

In [ ]:
from transformers import SpeechT5ForTextToSpeech

model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

In [ ]:
from functools import partial

# disable cache during training since it's incompatible with gradient checkpointing
model.config.use_cache = False

# set language and task for generation and re-enable cache
model.generate = partial(model.generate, use_cache=True)

In [ ]:
model_name = checkpoint.split("/")[-1]
finetuned_model_name = f"{model_name}-finetuned-nst-da"

In [ ]:
train_epochs = 20

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=finetuned_model_name,
    per_device_train_batch_size=32,
    auto_find_batch_size=True,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_ratio=0.1,
    num_train_epochs=train_epochs,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="epoch",
    per_device_eval_batch_size=32,
    save_strategy="epoch",
    logging_steps=5,
    save_total_limit=1,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    greater_is_better=False,
    label_names=["labels"],
    push_to_hub=False,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    tokenizer=processor,
)

In [ ]:
# trainer.train(resume_from_checkpoint=resume_from_checkpoint)
trainer.train()

In [ ]:
# NOTE evaluate explicitly so that best model performance is updated on model card
trainer.evaluate(dataset["test"])

In [ ]:
kwargs = {
    "dataset_tags": "JackismyShephard/nst-da-norm",
    "dataset": "NST Danish ASR Database Normalized",
    "model_name": finetuned_model_name,
    "finetuned_from": checkpoint,
    "tasks": "text-to-speech",
    "language": "da",
}

In [ ]:
trainer.push_to_hub(**kwargs)

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-to-speech",
    model="JackismyShephard/speecht5_tts-finetuned-nst-da",
    use_fast=True,
    device=0 if torch.cuda.is_available() else "cpu",
    revision="main",
)

In [ ]:
def replace_danish_letters(text):
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

In [ ]:
text = "I sin oprindelige før-kristne form blev alferne sandsynligvis opfattet som en personificering af det land og den natur, der omgav menneskene, dvs. den opdyrkede jord, gården og de naturressourcer, som hørte dertil. De var guddommelige eller delvis guddommelige væsener, der besad magiske kræfter, som de brugte både til fordel og ulempe for menneskene."
text = replace_danish_letters(text)
text2 = "Selvom mosser er almindelige, er en lang række mosser dog knyttet snævert til habitater der er i stærk tilbagegang og disse mosser er truede. Kendtest af disse er måske tørvemosserne hvor flere arter kun forekommer i såkaldte højmoser der nu er meget sjældne i Danmark."
text2 = replace_danish_letters(text2)

In [ ]:
root = "./embeddings/nst-da-metricgan-plus/"
speaker_embedding_path = root + "male_51_vest_sydsjaelland.npy"

In [ ]:
import numpy as np

speaker_embedding = np.load(speaker_embedding_path)
speaker_embedding = torch.tensor(speaker_embedding).unsqueeze(0)

In [ ]:
forward_params = {"speaker_embeddings": speaker_embedding}
speech_pipe = pipe(text2, forward_params=forward_params)

In [ ]:
speech_enhanced, new_sr = mega_enhance_audio(
    speech_pipe["audio"], speech_pipe["sampling_rate"]
)

In [ ]:
from IPython.display import Audio

Audio(speech_pipe["audio"], rate=speech_pipe["sampling_rate"])

In [ ]:
Audio(speech_enhanced, rate=new_sr)